In [36]:
import numpy as np
import pandas as pd
from pprint import pprint
from json import  dumps, dump

### Functions to maintain example database

In [57]:
# static variables (used for id's)
p_id:int = 0  # person id
g_id:int = 0  # group id
t_id:int = 0  # transaction id

# reset ids
def reset_ids():
    global p_id, g_id, t_id
    p_id = 0
    g_id = 0
    t_id = 0

# static collections
people:dict = {}
groups:dict = {}
transactions:dict = {}
items: dict = {}

# reset collections
def reset_collections():
    global people, groups, transactions, items
    people = {}
    groups = {}
    transactions = {}
    items = {}

def reset_db():
    reset_ids()
    reset_collections()

# helper function to print out example databases
def print_collections(name = 0):
    if type(name) is int:
        print(f'people:\n{dumps(people)}')
        print(f'groups:\n{dumps(groups)}')
        print(f'transactions:\n{dumps(transactions)}')
        print(f'items:\n{dumps(items)}')
        return
    if type(name) is str:
        name = [name]
    if type(name) is list:
        for i in name:
            if i == 'people':
                print(f'people:\n{dumps(people)}')
            elif i == 'groups':
                print(f'groups:\n{dumps(groups)}')
            elif i == 'transactions':
                print(f'transactions:\n{dumps(transactions)}')
            elif i == 'items':
                print(f'items:\n{dumps(items)}')


### Functions to create database objects

In [134]:
# create a basic person object
def create_person(first_name:str, last_name:str, email:str) -> dict:
    global p_id, people
    person = {
        'id': p_id,
        'first_name': first_name,
        'last_name': last_name,
        'email': email,
        'groups': []
    }
    people[p_id] = person

    p_id += 1
    return person

# create a basic group object
def create_group(name:str) -> dict:
    global g_id, groups
    group = {
        'id': g_id,
        'name': name,
        'people': [],
        'transactions': []
    }
    groups[g_id] = group

    g_id += 1
    return group

# create a basic transaction object
def create_transaction(description:str, p_id:int, g_id:int) -> dict:
    global t_id, transactions, groups

    # add person to group
    if g_id not in groups:
        print(f'ERROR: group id [{g_id}] does not exist')
        return None
    # check if person exists
    if p_id not in people:
        print(f'ERROR: person id [{t_id}] does not exist')
        return None
    # check if person is in group
    if p_id not in groups[g_id]['people']:
        print(f'ERROR: person id [{p_id}] is not in group [{g_id}]')
        return None

    transaction = {
        'id': t_id,
        'group': g_id,
        'purchaser': p_id,
        'description': description,
        'total_transaction_price': 0,
        'items': []
    }

    # add transaction to transactions collection
    transactions[t_id] = transaction

    # add transactions to group
    groups[g_id]['transactions'].append(t_id)
    t_id += 1
    return transaction

# create item
def create_item(description:str, price:float) -> dict:
    global items


    if price <= 0:
        print(f'ERROR: price [{price}] should be greater than 0')
        return None

    i_id = hash(f"{description},{price}")
    if i_id not in items:
        item = {
            'id': i_id,
            'price': price,
            'description': description,
        }
        items[i_id] = item
    else:
        item = items[i_id]
    return item


In [135]:
# getter methods

# get person given their email
def get_person(email:str) -> dict:
    """
    SELECT * FROM people WHERE people.email = email;
    """
    global people
    for id, person in people.items():
        if person['email'] == email:
            return person
    print(f'ERROR: email [{email}] does not exist')
    return None



In [136]:
# addition methods

# add person to group
def add_person_to_group(g_id:int, email:str):
    # todo decide if this should be email or id or name, etc
    global people, groups

    # get persons id and check if they exist
    person = get_person(email)
    if person is None:
        return
    p_id = person['id']

    # add person to group
    if g_id not in groups:
        print(f'ERROR: group id [{g_id}] does not exist')
        return
    groups[g_id]['people'].append(p_id)

    # add group to person
    people[p_id]['groups'].append(g_id)

def add_item_to_transaction(t_id:int, i_id:int, p_id:int, quantity:int):
    global transactions, items, people, groups
    # check if item exists
    if i_id not in items:
        print(f'ERROR: item id [{i_id}] does not exist')
        return
    # check if transaction exists
    if t_id not in transactions:
        print(f'ERROR: transaction id [{t_id}] does not exist')
        return
    # check if person exists
    if p_id not in people:
        print(f'ERROR: person id [{p_id}] does not exist')
        return
    # check if person is in transaction's group
    g_id = transactions[t_id]['group']
    if p_id not in groups[g_id]['people']:
        print(f'ERROR: person id [{p_id}] is not in group [{g_id}]')
        return
    # check quantity
    if quantity <= 0:
        print(f'ERROR: quantity [{quantity}] should be greater than 0')
        return

    total_item_price = quantity*items[i_id]['price']

    # create item entry
    item_entry = {
        'person': p_id,
        'item': i_id,
        'quantity': quantity,
        'total_item_price': total_item_price
    }

    # update transaction total price and add item to transaction
    transactions[t_id]['total_transaction_price'] += total_item_price
    transactions[t_id]['items'].append(item_entry)


In [141]:
# Calculations
def calculate_group_payout(g_id:int) -> dict:
    global groups, transactions
    # add person to group
    if g_id not in groups:
        print(f'ERROR: group id [{g_id}] does not exist')
        return None
    group = groups[g_id]

    # initialize all owed to 0
    payout = { person: 0 for person in group['people']}

    #
    for t_id in group['transactions']:
        transaction = transactions[t_id]
        purchaser = transaction['purchaser']
        payout[purchaser] -= transaction['total_transaction_price']
        for item in transaction['items']:
            payout[item['person']] += item['total_item_price']

    return payout

In [142]:
# Clear DB
reset_db()

# Create some people
hunter  = create_person('Hunter', 'Price', 'hprice7@vols.utk.edu')
andrei  = create_person('Andrei', 'Cozma', 'acozma@vols.utk.edu')
blake   = create_person('Blake', 'Childress', 'cchildr3@vols.utk.edu')
emanuel = create_person('Emanuel', 'Chavez', 'echavez2@vols.utk.edu')
jacob   = create_person('Jacob', 'Leonard', 'jleona19@vols.utk.edu')
lillian = create_person('Lillian', 'Coar', 'wcoar@vols.utk.edu')
tyler   = create_person('Tyler', 'Beichler', 'tbeichle@vols.utk.edu')
print_collections(['people'])
print('-'*50)

# Create some groups
group1 = create_group('group1')
group2 = create_group('group2')
group3 = create_group('group3')
group4 = create_group('group4')
print_collections(['groups'])
print('-'*50)

# Add people to groups
add_person_to_group(group1['id'], hunter['email'])
add_person_to_group(group1['id'], andrei['email'])
add_person_to_group(group1['id'], blake['email'])
add_person_to_group(group1['id'], emanuel['email'])

add_person_to_group(group2['id'], hunter['email'])
add_person_to_group(group2['id'], lillian['email'])

add_person_to_group(group3['id'], hunter['email'])
add_person_to_group(group3['id'], jacob['email'])
add_person_to_group(group3['id'], blake['email'])

add_person_to_group(group4['id'], hunter['email'])
add_person_to_group(group4['id'], andrei['email'])
add_person_to_group(group4['id'], blake['email'])
add_person_to_group(group4['id'], emanuel['email'])
add_person_to_group(group4['id'], jacob['email'])
add_person_to_group(group4['id'], lillian['email'])
add_person_to_group(group4['id'], tyler['email'])
print_collections(['people', 'groups'])
print('-'*50)

# create some transactions
t1 = create_transaction('Grocery Store1', hunter['id'], group1['id'])
t2 = create_transaction('Grocery Store2', lillian['id'], group4['id'])
t3 = create_transaction('Grocery Store3', andrei['id'], group4['id'])

# create some items
i1 = create_item('Milk', 6.99)
i2 = create_item('Eggs', 9.99)
i3 = create_item('Cheese', 3.99)
i4 = create_item('Bacon', 15.99)
i5 = create_item('Paper Towels', 19.99)

# connect items to transactions
add_item_to_transaction(t1['id'],i1['id'],hunter['id'], 3)
add_item_to_transaction(t1['id'],i2['id'],andrei['id'], 1)
add_item_to_transaction(t1['id'],i3['id'],andrei['id'], 2)
add_item_to_transaction(t1['id'],i4['id'],blake['id'], 1)

add_item_to_transaction(t2['id'],i2['id'],hunter['id'], 4)
add_item_to_transaction(t2['id'],i4['id'],jacob['id'], 1)
add_item_to_transaction(t2['id'],i5['id'],tyler['id'], 1)

add_item_to_transaction(t3['id'],i1['id'],emanuel['id'], 1)
add_item_to_transaction(t3['id'],i1['id'],andrei['id'], 1)

print_collections(['groups','transactions', 'items'])
print('-'*50)

people:
{"0": {"id": 0, "first_name": "Hunter", "last_name": "Price", "email": "hprice7@vols.utk.edu", "groups": []}, "1": {"id": 1, "first_name": "Andrei", "last_name": "Cozma", "email": "acozma@vols.utk.edu", "groups": []}, "2": {"id": 2, "first_name": "Blake", "last_name": "Childress", "email": "cchildr3@vols.utk.edu", "groups": []}, "3": {"id": 3, "first_name": "Emanuel", "last_name": "Chavez", "email": "echavez2@vols.utk.edu", "groups": []}, "4": {"id": 4, "first_name": "Jacob", "last_name": "Leonard", "email": "jleona19@vols.utk.edu", "groups": []}, "5": {"id": 5, "first_name": "Lillian", "last_name": "Coar", "email": "wcoar@vols.utk.edu", "groups": []}, "6": {"id": 6, "first_name": "Tyler", "last_name": "Beichler", "email": "tbeichle@vols.utk.edu", "groups": []}}
--------------------------------------------------
groups:
{"0": {"id": 0, "name": "group1", "people": [], "transactions": []}, "1": {"id": 1, "name": "group2", "people": [], "transactions": []}, "2": {"id": 2, "name": 

In [143]:
payout = calculate_group_payout(group4['id'])
pprint(payout)

{0: 39.96, 1: -6.99, 2: 0, 3: 6.99, 4: 15.99, 5: -75.94, 6: 19.99}
